Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [2]:
from random import choices

import lab9_lib

In [11]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{fitness(ind):.2%}")

print(fitness.calls)

5.83%
5.69%
5.37%
11.54%
5.36%
4.90%
16.77%
5.29%
10.60%
5.46%
10


In [134]:
from dataclasses import dataclass
from random import randint,choice, random
from copy import copy

fitness = lab9_lib.make_problem(10)

POPULATION_SIZE = 20
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = .3

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop): #seleziono il genitore da cui mutare
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, len(offspring.genotype)-1) 
    
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring

"""
we use a really big value for the generations (100k) in order to find 
an upperbound of the value of the fitness function without thinking about the
number of fitness calls
"""

for generation in range(10_000): 
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE] 
    print(population[0].fitness)


print(fitness.calls)
fitness._calls = 0

0.11381245783
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1592467906
0.1735805695
0.1735805695
0.1735805695
0.1737796805
0.1737796805
0.17378068049999998
0.17378068049999998
0.17378967950000002
0.1737907795
0.1738897795
0.1738907795
0.1739007795
0.1739007805
0.1739007805
0.1740017795
0.1740017795
0.1740017795
0.1740117795
0.1740127895
0.1740127895
0.1740127895
0.17411178949999997
0.1741126896
0.1741226896
0.1741236906
0.1741236906
0.1742236906
0.1742236906
0.1742236906
0.1742246907
0.1742256906
0.1742356916
0.1742357006
0.17423670060000002
0.17423670060000002
0.1743357006
0.1743458006
0.1743458006
0.1744458006
0.1744458006
0.17445580060000002
0.17445580060000002
0.1745558006
0.1745568006
0.1745568006
0.1745568006
0.1745568006
0.1745568006
0.1745667906
0.1745668906
0.1746667906
0.1746667906
0.17466779059999998
0.1746678906


In [135]:
from dataclasses import dataclass
from random import randint,choice, random
from copy import copy

#fitness = lab9_lib.make_problem(10)

POPULATION_SIZE = 20
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = .3

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop): #seleziono il genitore da cui mutare
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def most_diverse_couple(k=POPULATION_SIZE//2):
    best_couple = None
    best_diversity = -1
    for i in range(k):
        #select a random couple of parents
        p1, p2 = choice(population), choice(population)
        diversity_value = diversity(p1, p2)
        if diversity_value > best_diversity:
            best_couple = p1, p2
            best_diversity = diversity_value
    return best_couple

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, len(offspring.genotype)-1) 
    
    offspring.genotype[pos] = 1-offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring


def diversity(ind1: Individual, ind2: Individual):
    diff = 0.0
    for i in range(0, len(ind1.genotype)):
        if ind1.genotype[i] != ind2.genotype[i]:
            diff = diff + 1
    return float(diff)/float(len(ind1.genotype))


"""
we use a really big value for the generations (100k) in order to find 
an upperbound of the value of the fitness function without thinking about the
number of fitness calls
"""

for generation in range(8_000): 
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            #p1 = select_parent(population)
            #p2 = select_parent(population)
            p1, p2 = most_diverse_couple()
            """
            print("Generation #{}".format(counter))
            print(p1)
            print(p2)
            """
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE] 
    print(population[0].fitness)


print(fitness.calls)
fitness._calls = 0
#diversity(population[0], population[19])
#<print(population[0])
#print(population[19])

0.10781124585
0.10781124585
0.10781124585
0.11188934484
0.11188934484
0.1648003895
0.1709023454
0.1709023454
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214024799
0.214134798
0.214134798
0.214134798
0.214134798
0.214134798
0.214134798
0.214134798
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.225934455
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434785
0.27434

In [ ]:
from dataclasses import dataclass
from random import randint,choice, random, sample
from copy import copy

#fitness = lab9_lib.make_problem(10)

n_mutation = 1
POPULATION_SIZE = 20
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = .3

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop): #seleziono il genitore da cui mutare
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def most_diverse_couple(k=POPULATION_SIZE//2):
    best_couple = None
    best_diversity = -1
    for i in range(k):
        #select a random couple of parents
        p1, p2 = choice(population), choice(population)
        diversity_value = diversity(p1, p2)
        if diversity_value > best_diversity:
            best_couple = p1, p2
            best_diversity = diversity_value
    return best_couple

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = sample(range(1000), k=n_mutation) 
    for p in pos:
        offspring.genotype[p] = 1-offspring.genotype[p]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring


def diversity(ind1: Individual, ind2: Individual):
    diff = 0.0
    for i in range(0, len(ind1.genotype)):
        if ind1.genotype[i] != ind2.genotype[i]:
            diff = diff + 1
    return float(diff)/float(len(ind1.genotype))


"""
we use a really big value for the generations (100k) in order to find 
an upperbound of the value of the fitness function without thinking about the
number of fitness calls
"""
previous_fitness=-1
counter_same_fitness = 0
for generation in range(8_000): 
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            #p1 = select_parent(population)
            #p2 = select_parent(population)
            p1, p2 = most_diverse_couple()
            """
            print("Generation #{}".format(counter))
            print(p1)
            print(p2)
            """
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE] 
    
    
    if population[0].fitness>previous_fitness:
        previous_fitness = population[0].fitness
        counter_same_fitness = 0
        #n_mutation = max(1, n_mutation-1)
        n_mutation = 1
    else:
        counter_same_fitness += 1
        if counter_same_fitness >= 5 : #after 5 same fitness values we increare the N_MUTATION value
            n_mutation = min(100, n_mutation+1)
    print(f"Generation {generation} is doing {n_mutation} mutations")
    print(population[0].fitness)

print(fitness.calls)
fitness.calls = 0
#diversity(population[0], population[19])
#<print(population[0])
#print(population[19])

Generation 0 is doing 1 mutations
0.1622345686
Generation 1 is doing 1 mutations
0.1707014709
Generation 2 is doing 1 mutations
0.1707014709
Generation 3 is doing 1 mutations
0.17070157090000002
Generation 4 is doing 1 mutations
0.17070157090000002
Generation 5 is doing 1 mutations
0.17070157090000002
Generation 6 is doing 1 mutations
0.17070157090000002
Generation 7 is doing 1 mutations
0.17070157090000002
Generation 8 is doing 2 mutations
0.17070157090000002
Generation 9 is doing 3 mutations
0.17070157090000002
Generation 10 is doing 4 mutations
0.17070157090000002
Generation 11 is doing 5 mutations
0.17070157090000002
Generation 12 is doing 6 mutations
0.17070157090000002
Generation 13 is doing 7 mutations
0.17070157090000002
Generation 14 is doing 8 mutations
0.17070157090000002
Generation 15 is doing 9 mutations
0.17070157090000002
Generation 16 is doing 10 mutations
0.17070157090000002
Generation 17 is doing 11 mutations
0.17070157090000002
Generation 18 is doing 12 mutations
0.1

KeyboardInterrupt: 

In [ ]:
from dataclasses import dataclass
from random import randint,choice, random, sample
from copy import copy

#fitness = lab9_lib.make_problem(10)

n_mutation = 1
POPULATION_SIZE = 20
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = .3

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop): #seleziono il genitore da cui mutare
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def most_diverse_couple(k=POPULATION_SIZE//2):
    best_couple = None
    best_diversity = -1
    for i in range(k):
        #select a random couple of parents
        p1, p2 = choice(population), choice(population)
        diversity_value = diversity(p1, p2)
        if diversity_value > best_diversity:
            best_couple = p1, p2
            best_diversity = diversity_value
    return best_couple

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = sample(range(1000), k=n_mutation) 
    for p in pos:
        offspring.genotype[p] = 1-offspring.genotype[p]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring


def diversity(ind1: Individual, ind2: Individual):
    diff = 0.0
    for i in range(0, len(ind1.genotype)):
        if ind1.genotype[i] != ind2.genotype[i]:
            diff = diff + 1
    return float(diff)/float(len(ind1.genotype))


"""
we use a really big value for the generations (100k) in order to find 
an upperbound of the value of the fitness function without thinking about the
number of fitness calls
"""
previous_fitness=-1
counter_same_fitness = 0
for generation in range(8_000): 
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            #p1 = select_parent(population)
            #p2 = select_parent(population)
            p1, p2 = most_diverse_couple()
            """
            print("Generation #{}".format(counter))
            print(p1)
            print(p2)
            """
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE] 
    
    
    if population[0].fitness>previous_fitness:
        previous_fitness = population[0].fitness
        counter_same_fitness = 0
        #n_mutation = max(1, n_mutation-1)
        n_mutation = 1
    else:
        counter_same_fitness += 1
        if counter_same_fitness >= 150 : #after 5 same fitness values we increare the N_MUTATION value
            n_mutation = min(200, n_mutation+1)
    print(f"Generation {generation} is doing {n_mutation} mutations")
    print(population[0].fitness)

print(fitness.calls)
fitness.calls = 0
#diversity(population[0], population[19])
#<print(population[0])
#print(population[19])

Generation 0 is doing 1 mutations
0.10790457952
Generation 1 is doing 1 mutations
0.15903678070000002
Generation 2 is doing 1 mutations
0.15903678070000002
Generation 3 is doing 1 mutations
0.15903678070000002
Generation 4 is doing 1 mutations
0.15903678070000002
Generation 5 is doing 1 mutations
0.214457806
Generation 6 is doing 1 mutations
0.214457806
Generation 7 is doing 1 mutations
0.214457806
Generation 8 is doing 1 mutations
0.214457806
Generation 9 is doing 1 mutations
0.214457806
Generation 10 is doing 1 mutations
0.214457806
Generation 11 is doing 1 mutations
0.21446681799999998
Generation 12 is doing 1 mutations
0.21446681799999998
Generation 13 is doing 1 mutations
0.21446681799999998
Generation 14 is doing 1 mutations
0.21446681799999998
Generation 15 is doing 1 mutations
0.21446681799999998
Generation 16 is doing 1 mutations
0.21446681799999998
Generation 17 is doing 1 mutations
0.21446681799999998
Generation 18 is doing 1 mutations
0.21446681799999998
Generation 19 is do